# Домашняя работа
Взять boston house-prices datase (sklearn.datasets.load_boston) и сделать тоже самое для задачи регрессии (попробовать разные алгоритмы, поподбирать параметры, вывести итоговое качество)

In [59]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import PolynomialFeatures

import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.DataFrame(data = load_boston()['data'], columns = load_boston()['feature_names'])
data

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48


In [3]:
target = load_boston()['target']
len(target)

506

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
dtypes: float64(13)
memory usage: 51.5 KB


In [102]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=42)

Опыт работы с бостонским датасетом позволяет предположить, что генерирование полиномиальных признаков улучшит модель. Поэтому сразу сделаю вариант матрицы признаков с полиномиальными признаками

In [77]:
data_poly = PolynomialFeatures(degree=2).fit_transform(data)

In [103]:
Xp_train, Xp_test, yp_train, yp_test = train_test_split(data_poly, target, test_size=0.3, random_state=42)

Буду тестировать три алгоритма: RandomForestRegressor, KNeighborsRegressor и Lasso. Для каждого из них создам словарь с вариантами параметров. 

In [170]:
forest_params = {
    'n_estimators': range(5, 200, 5),
    'criterion': ('squared_error', 'absolute_error', 'poisson'),
    'max_depth': list(range(1, 30)) + [None],
    'max_features': ['auto', 'sqrt', 'log2', 25]
}

In [6]:
neighbors_params = {
    'n_neighbors': range(1, 35),
    'weights': ('uniform', 'distance'),
    'algorithm': ('auto', 'ball_tree', 'kd_tree', 'brute'),
    'p': range(1, 6)
}

In [60]:
lasso_params = {
    'alpha': list(2**np.linspace(-10,10,100)),
    'fit_intercept': (True, False),
    'normalize': (True, False),
    'warm_start':(True, False),
    'positive': (True, False),
    'selection': ('cyclic', 'random')
}

GridSearchCV для RandomForest работает слишком долго, поэтому заменю его RandomizedSearchCV. oob_score ставлю равным True, чтобы минимизировать переобучение. К значению cv=25 пришла опытным путем: это компромисс между временем работы и качеством.

In [171]:
forest_grid = RandomizedSearchCV(
    RandomForestRegressor(oob_score=True),
    forest_params,
    cv=25,
    scoring = 'neg_root_mean_squared_error
)

In [172]:
%%time

for _ in range(5):
    forest_grid.fit(data, target)
    print('*' * 40)
    print(forest_grid.best_estimator_)
    print(forest_grid.best_score_)

****************************************
RandomForestRegressor(criterion='absolute_error', max_features='log2',
                      n_estimators=55, oob_score=True)
-3.627993675108557
****************************************
RandomForestRegressor(max_depth=19, max_features='sqrt', n_estimators=65,
                      oob_score=True)
-3.503290337339582
****************************************
RandomForestRegressor(criterion='absolute_error', max_depth=10,
                      max_features='log2', n_estimators=160, oob_score=True)
-3.511013652276719
****************************************
RandomForestRegressor(criterion='absolute_error', max_depth=13,
                      max_features='log2', n_estimators=115, oob_score=True)
-3.4922771144943128
****************************************
RandomForestRegressor(criterion='absolute_error', max_depth=10,
                      max_features='sqrt', n_estimators=120, oob_score=True)
-3.5514796741945758
Wall time: 9min 48s


In [176]:
mean_squared_error(forest_grid.predict(X_train), y_train)

2.1058726616446943

In [177]:
mean_squared_error(forest_grid.predict(X_test), y_test)

2.1971199401498525

Предобученный алгоритм дал хорошие результаты на тренировочной и тестовой выборках. Теперь я подставлю найденные параметры в необученный RandomForestRegressor и проверю его результаты

In [178]:
forest_control = RandomForestRegressor(criterion='absolute_error', max_depth=13,
                      max_features='log2', n_estimators=115, oob_score=True)
forest_control.fit(X_train, y_train)
mean_squared_error(forest_control.predict(X_test), y_test)

9.840166053129039

In [179]:
mean_squared_error(forest_control.predict(X_train), y_train)

1.9761330273514646

Модель переобучилась, но при oob_score = False все было намного хуже.

Та же последовательность действий для данных с полиномиальными признаками

In [180]:
forest_grid_poly = RandomizedSearchCV(
    RandomForestRegressor(oob_score=True),
    forest_params,
    cv=25,
    scoring = 'neg_root_mean_squared_error'
)

In [181]:
%%time

for _ in range(5):
    forest_grid_poly.fit(data_poly, target)
    print('*' * 40)
    print(forest_grid_poly.best_estimator_)
    print(forest_grid_poly.best_score_)

****************************************
RandomForestRegressor(criterion='absolute_error', max_depth=14,
                      max_features='sqrt', n_estimators=185, oob_score=True)
-3.4444464946534947
****************************************
RandomForestRegressor(max_depth=16, max_features=25, n_estimators=20,
                      oob_score=True)
-3.461770340541142
****************************************
RandomForestRegressor(criterion='absolute_error', max_depth=15, max_features=25,
                      n_estimators=130, oob_score=True)
-3.3905238938486666
****************************************
RandomForestRegressor(max_depth=21, max_features=25, n_estimators=120,
                      oob_score=True)
-3.423576983568042
****************************************
RandomForestRegressor(criterion='absolute_error', max_depth=24, max_features=25,
                      n_estimators=150, oob_score=True)
-3.4085598300061464
Wall time: 38min 17s


In [183]:
mean_squared_error(forest_grid_poly.predict(Xp_train), yp_train)

1.436245748901444

In [184]:
mean_squared_error(forest_grid_poly.predict(Xp_test), yp_test)

1.312329814327489

In [185]:
forest_control_poly = RandomForestRegressor(criterion='absolute_error', max_depth=15, max_features=25,
                      n_estimators=130, oob_score=True)
forest_control_poly.fit(Xp_train, yp_train)
mean_squared_error(forest_control_poly.predict(Xp_test), yp_test)

8.840799918249761

In [186]:
mean_squared_error(forest_control_poly.predict(Xp_train), yp_train)

1.6794601665663746

Ошибка на тестовой выборке уменьшилась на единицу по сравнению с моделью, обученной без полиномиальных признаков. Разрыв между тренировочной и тестовой выборкой тоже уменьшился. Модель все равно переобучается, но прогресс заметен.

Для KNeighborsRegressor можно использовать GridSearchCV. Чтобы избежать переобучения, я довела cv до 50. Это помогло уменьшить ошибку, но переобучение все равно очень сильное.

In [155]:
%%time

neighbors_best = GridSearchCV(KNeighborsRegressor(), neighbors_params, cv=50, scoring = 'neg_root_mean_squared_error')
neighbors_best.fit(data, target)

Wall time: 6min 18s


GridSearchCV(cv=50, estimator=KNeighborsRegressor(),
             param_grid={'algorithm': ('auto', 'ball_tree', 'kd_tree', 'brute'),
                         'n_neighbors': range(1, 35), 'p': range(1, 6),
                         'weights': ('uniform', 'distance')},
             scoring='neg_root_mean_squared_error')

In [156]:
neighbors_best.best_estimator_

KNeighborsRegressor(n_neighbors=6, p=1, weights='distance')

In [157]:
neighbors_best.best_score_

-5.728238164863335

In [158]:
mean_squared_error(neighbors_best.predict(X_train), y_train)

0.0

In [159]:
mean_squared_error(neighbors_best.predict(X_test), y_test)

0.0

In [160]:
neighbors_control = KNeighborsRegressor(n_neighbors=6, p=1, weights='distance')
neighbors_control.fit(X_train, y_train)
mean_squared_error(neighbors_control.predict(X_test), y_test)

22.389989574113468

In [161]:
mean_squared_error(neighbors_control.predict(X_train), y_train)

0.0

Использование полиномиальных признаков в целом не давало существенных результатов для KNeighborsRegressor, поэтому я не стала запускать GridSearchCV с cv=50, а оставила один из первоначальных вариантов. Результаты те же: при контрольном запуске большая MSE на тестовой выборке и нулевая на тренировочной, то есть переобучение.

In [111]:
%%time

neighbors_best_poly = GridSearchCV(KNeighborsRegressor(), neighbors_params, cv=15, scoring = 'neg_root_mean_squared_error')
neighbors_best_poly.fit(data_poly, target)

Wall time: 8min 43s


GridSearchCV(cv=15, estimator=KNeighborsRegressor(),
             param_grid={'algorithm': ('auto', 'ball_tree', 'kd_tree', 'brute'),
                         'n_neighbors': range(1, 35), 'p': range(1, 6),
                         'weights': ('uniform', 'distance')},
             scoring='neg_root_mean_squared_error')

In [112]:
neighbors_best_poly.best_estimator_

KNeighborsRegressor(n_neighbors=15, p=1, weights='distance')

In [115]:
neighbors_best_poly.best_score_

-7.670923029161531

In [114]:
mean_squared_error(neighbors_best_poly.predict(Xp_train), yp_train)

0.0

In [116]:
mean_squared_error(neighbors_best_poly.predict(Xp_test), yp_test)

0.0

In [121]:
neighbors_control_poly = KNeighborsRegressor(n_neighbors=15, p=1, weights='distance')
neighbors_control_poly.fit(Xp_train, yp_train)
mean_squared_error(neighbors_control_poly.predict(Xp_test), yp_test)

30.614767304855953

In [122]:
mean_squared_error(neighbors_control_poly.predict(Xp_train), yp_train)

0.0

Lasso - единственная модель, которая почти не переобучилась. Без полиномиальных признаков она дает огромную MSE - 23 на тестовой выборке, но зато MSE тестовой и тренировочной выборки отличаются всего на единицу.

In [78]:
%%time

lasso_best = GridSearchCV(Lasso(), lasso_params, cv=10, scoring = 'neg_root_mean_squared_error')
lasso_best.fit(data, target)

Wall time: 2min 11s


GridSearchCV(cv=10, estimator=Lasso(),
             param_grid={'alpha': [0.0009765625, 0.0011233476571973669,
                                   0.0012921957979451526, 0.0014864231651962546,
                                   0.001709844459752558, 0.001966847762467772,
                                   0.0022624807178568238, 0.0026025496717912854,
                                   0.0029937337103836327, 0.0034437158398282082,
                                   0.003961333883621868, 0.004556754060844206,
                                   0.005241670654642098, 0.006029535692485006...
                                   0.01606877812483144, 0.01848404404280697,
                                   0.021262343752724643, 0.024458244138135844,
                                   0.0281345139217778, 0.03236335646762575,
                                   0.037227827882957455, 0.042823468272503856,
                                   0.049260178183150566, 0.05666437709329101, ...],
            

In [79]:
lasso_best.best_estimator_

Lasso(alpha=0.042823468272503856, fit_intercept=False, normalize=True,
      selection='random')

In [80]:
lasso_best.best_score_

-4.880332626127531

In [81]:
mean_squared_error(lasso_best.predict(X_train), y_train)

24.73854013149876

In [82]:
mean_squared_error(lasso_best.predict(X_test), y_test)

23.062051790165942

In [83]:
lasso_control = Lasso(alpha=0.042823468272503856, fit_intercept=False, normalize=True,
      selection='random')
lasso_control.fit(X_train, y_train)
mean_squared_error(lasso_control.predict(X_test), y_test)

24.92132762358376

In [84]:
mean_squared_error(lasso_control.predict(X_train), y_train)

24.40358169732002

Использование полиномиальных признаков позволило уменьшить MSE примерно с 24 до 10, причем переобучение минимально. Это лучший результат среди всех использованных алгоритмов.

In [123]:
%%time

lasso_best_poly = GridSearchCV(Lasso(), lasso_params, cv=10, scoring = 'neg_root_mean_squared_error')
lasso_best_poly.fit(data_poly, target)

Wall time: 7min 48s


GridSearchCV(cv=10, estimator=Lasso(),
             param_grid={'alpha': [0.0009765625, 0.0011233476571973669,
                                   0.0012921957979451526, 0.0014864231651962546,
                                   0.001709844459752558, 0.001966847762467772,
                                   0.0022624807178568238, 0.0026025496717912854,
                                   0.0029937337103836327, 0.0034437158398282082,
                                   0.003961333883621868, 0.004556754060844206,
                                   0.005241670654642098, 0.006029535692485006...
                                   0.01606877812483144, 0.01848404404280697,
                                   0.021262343752724643, 0.024458244138135844,
                                   0.0281345139217778, 0.03236335646762575,
                                   0.037227827882957455, 0.042823468272503856,
                                   0.049260178183150566, 0.05666437709329101, ...],
            

In [124]:
lasso_best_poly.best_estimator_

Lasso(alpha=0.001966847762467772, normalize=True, warm_start=True)

In [125]:
lasso_best_poly.best_score_

-4.143924534813032

In [126]:
mean_squared_error(lasso_best_poly.predict(Xp_train), yp_train)

11.336372105178969

In [127]:
mean_squared_error(lasso_best_poly.predict(Xp_test), yp_test)

10.433489469253653

In [130]:
lasso_control_poly = Lasso(alpha=0.001966847762467772, normalize=True, warm_start=True)
lasso_control_poly.fit(Xp_train, yp_train)
mean_squared_error(lasso_control_poly.predict(Xp_test), yp_test)

12.381923662034767

In [131]:
mean_squared_error(lasso_control_poly.predict(Xp_train), yp_train)

9.790090209486118